# Import

In [ ]:
import pandas as pd

import importlib
import dataframes_util as ut
importlib.reload(ut)

# Concatenate all dataframes

In [ ]:
dfPosts = ut.read_all_csv("data/**/tweets.csv")

In [ ]:
dfPosts.info()

# Adding features to post dataframe

## Adding info about user error

In [ ]:
dfUsersErrors = ut.read_all_csv("data/**/not_found_users/users_not_found_inspected.csv")
dfPosts['user_code_error'] = float('nan')

for i,r in dfUsersErrors.iterrows():
    #User
    mask = dfPosts['user_id'] == r['id']
    dfPosts.loc[mask,'user_code_error'] = r['code']
    
    #Retweeted user
    mask = dfPosts['rt_user_id'] == r['id']
    dfPosts.loc[mask,'rt_user_code_error'] = r['code']
    
    #Replied users
    mask = dfPosts['in_reply_to_user_id'] == r['id']
    dfPosts.loc[mask,'in_reply_to_user_code_error'] = r['code']

In [ ]:
dfPosts

## Adding info about self-retweet

In [ ]:
dfPosts['is_self_rt'] = dfPosts['user_id'] == dfPosts['rt_user_id']

## Adding info about url credibility

In [ ]:
tweets_with_urls = pd.read_csv('data/tweets_with_urls_and_domain_and_score.csv',low_memory=False,lineterminator='\n',
                              usecols=['user_id','number_of_urls','HIGH','LOW','UNKW','SOCIAL'])
usersCred = ut.create_cred_score(tweets_with_urls)

In [ ]:
dfPosts['user_url_cred'] = float('nan')
dfPosts['rt_user_url_cred'] = float('nan')
dfPosts['in_reply_to_user_url_cred'] = float('nan')


for user_id,score in usersCred:    
    #User
    mask = dfPosts['user_id'] == user_id
    dfPosts.loc[mask,'user_url_cred'] = score
    
    #Retweeted user
    mask = dfPosts['rt_user_id'] == user_id
    dfPosts.loc[mask,'rt_user_url_cred'] = score
    
    #Replied users
    mask = dfPosts['in_reply_to_user_id'] == user_id
    dfPosts.loc[mask,'in_reply_to_user_url_cred'] = score

In [ ]:
dfPosts

# Write to file .csv the complete dataframes

In [ ]:
dfPosts.to_csv('../tweets.csv', index=False) 